In [21]:
# Use this cell to define your strategy

# final strategy:
import time
import math

ROWS = 6
COLS = 7
MOVE_ORDER = (3, 2, 4, 1, 5, 0, 6)

# precompute all winning 4-cell windows in the 7*6 board (col,row) 
WINDOWS = []
# horizontal
for r in range(ROWS):
    for c in range(COLS - 3):
        WINDOWS.append(((c, r), (c+1, r), (c+2, r), (c+3, r)))
# vertical
for c in range(COLS):
    for r in range(ROWS - 3):
        WINDOWS.append(((c, r), (c, r+1), (c, r+2), (c, r+3)))
# diag /
for c in range(COLS - 3):
    for r in range(ROWS - 3):
        WINDOWS.append(((c, r), (c+1, r+1), (c+2, r+2), (c+3, r+3)))
# diag \
for c in range(COLS - 3):
    for r in range(3, ROWS):
        WINDOWS.append(((c, r), (c+1, r-1), (c+2, r-2), (c+3, r-3)))


def my_strategy(board, player_number):
    '''Write your strategy here
    (param) board ([[int]*6]*7): The board the move will be taken on. For each position, will be 0 for unoccupied, 1 for occupied by player 1, 2 for occupied by player 2.
    (param) int player_number The number of your pieces in the board
    (return) int the number of the column you want your next move to be in (0 for the leftmost column, 6 for the rightmost)'''
    
    """
    board[col][row], row=0 bottom, values 0/1/2.
    Must return an int 0-6 for a non-full column.
    """
    start = time.perf_counter()

    TIME_BUDGET = 0.12  # tune: 0.08–0.20 depending on scoring

    def time_up():
        return (time.perf_counter() - start) >= TIME_BUDGET

    # always compute a legal fallback first (prevent forfeit)
    try:
        heights = compute_heights(board)
        legal = legal_moves(heights)            # define legal moves on this turn (can be reused for tactics and search)
        fallback = legal[0] if legal else 0     # define fallback from legal moves, or 0 if no legal moves (should be game over)
    except Exception:
        return 0

    try:
        me = 1 if int(player_number) == 1 else 2
        opp = 2 if me == 1 else 1

        # first move: center
        if sum(heights) == 0:
            return 3 if heights[3] < ROWS else fallback

        # tactical: win now / block now
        w = immediate_win(board, heights, me)
        if w is not None:
            return w
        b = immediate_win(board, heights, opp)
        if b is not None:
            return b

        # root anti-blunder: avoid moves that allow opponent immediate win
        candidates = [] # safe moves, ordered (center first)
        for c in ordered(legal):    # search ordered moves (center first)
            r = make_move(board, heights, c, me)    # return row where piece was placed
            lose = (immediate_win(board, heights, opp) is not None) # True if r is blunder, False if r is safe
            undo_move(board, heights, c, r) # undo move in altered board (for search)
            if not lose:    # if r is safe, add to candidates
                candidates.append(c)
        if not candidates:  # if no safe moves, will have to blunder, but still play legal center first moves
            candidates = ordered(legal)

        # search depth (tune for speed vs. strength)
        # good move ordering and pruning allow efficient search deeper than 4 in most cases
        MAX_DEPTH = 4   # -> 4 is a good starting point (lower for faster play, higher for stronger play)

        best_col = candidates[0]    # close to center, but overridden if better move found by search
        best_val = -math.inf    # negamax best value is negative infinity

        # Alpha-beta root: search optimization via pruning 
        alpha = -math.inf   # initial alpha for root (maximising for us, but negamax returns negative value, so initialised at -inf)
        beta = math.inf  # initial beta for root (minimising for opponent, but negamax returns negative value, so initialised at +inf)

        # root loop: search candidates with negamax + alpha-beta, but break if time is up (tune frequency of time checks by depth)
        for c in candidates:
            if time_up():   # time check 
                break
            r = make_move(board, heights, c, me)    # try move
            if win_from_last(board, c, r, me):  # check is immediate win within search
                undo_move(board, heights, c, r)
                return c
            val = -negamax(board, heights, MAX_DEPTH - 1, -beta, -alpha,
                           to_move=opp, other=me, root_player=me,
                           time_up=time_up, last_move=(c, r, me))   # recursive negamax search with alpha-beta pruning
                # next player is opponent (negamax sign flip), other is us, root_player is us for evaluation, time_up function passed down for time checks, last_move is move just made for win check in negamax
            undo_move(board, heights, c, r)

            if val > best_val:  # update best move if better value found
                best_val = val
                best_col = c
            if val > alpha:  # update alpha for root (best guaranteed score for us - alpha-beta pruning)
                alpha = val

        return ensure_valid(best_col, heights)

    except Exception:
        # Never forfeit on exception
        return fallback


# -------------------------
# Fast negamax + alpha-beta search
# -------------------------

def negamax(board, heights, depth, alpha, beta, to_move, other, root_player, time_up, last_move):
    if time_up():
        return evaluate(board, heights, root_player)

    lc, lr, lp = last_move  
    if win_from_last(board, lc, lr, lp):
        # last player just won => current player is losing
        return -10_000_000 + (5 - depth)    # "mate distance", prefer faster wins and slower losses

    if sum(heights) == ROWS * COLS: # board full case is a draw, no more legal moves
        return 0

    if depth == 0:  # base case: return heuristic evaluation of position for root player (search depth exhausted)
        return evaluate(board, heights, root_player)

    best = -math.inf    # best value for last player (negamax returns negative value, so initialised at -inf)
    legal = legal_moves(heights)    # get legal moves for current position (reused for move ordering in deeper search)
    if not legal:
        return 0

    for c in ordered(legal):    # ordered for move consistency with center first
        if time_up():   # time check before each move
            break
        r = make_move(board, heights, c, to_move)
        if win_from_last(board, c, r, to_move):
            # immediate win for current player => terrible for last player after negamax sign flip
            val = 10_000_000 - (5 - depth)
        else:   # recursive negamax search with alpha-beta pruning
            val = -negamax(board, heights, depth - 1, -beta, -alpha,
                           to_move=other, other=to_move, root_player=root_player,
                           time_up=time_up, last_move=(c, r, to_move))  # to_move and other take turns (c,r is the move just made)
        undo_move(board, heights, c, r) # recursive undo move in altered board

        if val > best:
            best = val
        if val > alpha:
            alpha = val
        if alpha >= beta:   # alpha-beta pruning (no need to search further if opponent can already do better than our best guaranteed score)
            break

    return best


# -------------------------
# Tactics / legality
# -------------------------

# board state check (at beginning of each turn)
def compute_heights(board):
    h = [0] * COLS
    for c in range(COLS):
        col = board[c]
        i = 0
        while i < ROWS and col[i] != 0:
            i += 1
        h[c] = i
    return h

# return list of legal moves 
def legal_moves(heights):
    return [c for c in range(COLS) if heights[c] < ROWS]

# center first move ordering 
def ordered(moves):
    s = set(moves)
    return [c for c in MOVE_ORDER if c in s]

# ensure move is legal, or return fallback (prevent forfeit)
def ensure_valid(col, heights):
    try:
        col = int(col)
    except Exception:
        col = 3
    if 0 <= col < COLS and heights[col] < ROWS:
        return col
    m = legal_moves(heights)
    return m[0] if m else 0

# simulate move (return row where piece was placed)
def make_move(board, heights, col, player):
    r = heights[col]
    board[col][r] = player  # global board is modified (will need to undo after search)
    heights[col] = r + 1
    return r

# undo move in altered board (for search)
def undo_move(board, heights, col, row):
    board[col][row] = 0
    heights[col] = row

# check for immediate win for player (return winning column, or None if no immediate win)
def immediate_win(board, heights, player):
    for c in ordered(legal_moves(heights)):
        r = make_move(board, heights, c, player)
        won = win_from_last(board, c, r, player)    # check if move wins
        undo_move(board, heights, c, r) # undo move in altered board 
        if won:
            return c
    return None


# -------------------------
# Win check around last move (fast)
# -------------------------

# check if last move at (c,r) by player resulted in a win by counting consecutive pieces in all 4 directions
def win_from_last(board, c, r, player):
    return (
        count_dir(board, c, r, 1, 0, player) + count_dir(board, c, r, -1, 0, player) >= 3 or
        count_dir(board, c, r, 0, 1, player) + count_dir(board, c, r, 0, -1, player) >= 3 or
        count_dir(board, c, r, 1, 1, player) + count_dir(board, c, r, -1, -1, player) >= 3 or
        count_dir(board, c, r, 1, -1, player) + count_dir(board, c, r, -1, 1, player) >= 3
    )

# look in direction (dc, dr) from (c,r) and count consecutive pieces for player until hitting edge of board or different piece
def count_dir(board, c, r, dc, dr, player):
    n = 0
    cc = c + dc
    rr = r + dr
    while 0 <= cc < COLS and 0 <= rr < ROWS and board[cc][rr] == player:
        n += 1
        cc += dc
        rr += dr
    return n


# -------------------------
# Window heuristic (gravity-aware)
# ------------------------

# heuristic evaluation: window scoring (blocked ignored) + playable-threat weighting
def evaluate(board, heights, me):
    opp = 2 if me == 1 else 1
    score = 0

    # bonus score for center control
    score += 6 * sum(1 for r in range(ROWS) if board[3][r] == me)   

    for (a, b, c, d) in WINDOWS:
        v0 = board[a[0]][a[1]]
        v1 = board[b[0]][b[1]]
        v2 = board[c[0]][c[1]]
        v3 = board[d[0]][d[1]]

        mc = (v0 == me) + (v1 == me) + (v2 == me) + (v3 == me)
        oc = (v0 == opp) + (v1 == opp) + (v2 == opp) + (v3 == opp)
        if mc and oc:   # if both players have pieces in window, it's blocked and no longer relevant for evaluation
            continue

        empties = 4 - mc - oc   # number of empty cells in window
        if mc == 4:
            score += 100000     # arbitrary weight for winning windows
            continue
        if oc == 4:
            score -= 100000     # arbitrary weight for opponent winning windows
            continue

        # playable empties: (col,row) is playable if row == heights[col] (gravity-aware)
        pe = 0
        if v0 == 0 and a[1] == heights[a[0]]: pe += 1
        if v1 == 0 and b[1] == heights[b[0]]: pe += 1
        if v2 == 0 and c[1] == heights[c[0]]: pe += 1
        if v3 == 0 and d[1] == heights[d[0]]: pe += 1

        # more pieces and more playable empties in window increases score for us, decreases score for opponent
        if mc == 3 and empties == 1:
            score += 120 if pe == 1 else 30
        elif oc == 3 and empties == 1:
            score -= 140 if pe == 1 else 45
        elif mc == 2 and empties == 2:
            score += 10 + 2 * pe
        elif oc == 2 and empties == 2:
            score -= 12 + 2 * pe

    return score

In [ ]:
# When you want to play against your strategy, run the cell above, then this cell
from game.run_game import run_game
from strategies import manual

run_game("Your Strategy", "You", my_strategy, manual.manual, move_duration=0.1)

In [ ]:
# Run this cell to see your code play a single match against the opponent you will face in the assessment
# Press enter to progress the game
from game.run_game import run_game
from assessment.assessor import random_opponent

run_game("Your Strategy", "Opponent", my_strategy, my_strategy_slow, move_duration=-1)

In [22]:
# When you're ready to run your strategy run the top cell, then this cell
# You can do this as often as you like as you improve your strategy
from assessment.assessor import assess

assess(my_strategy)

                                                                                                    
Results
Wins: 1000
Draws: 0
Losses: 0
Forfeits: 0
Mark: 100.0%
